
# __Algoritmo de Dijkstra — Caminho Mínimo em Grafos Ponderados Positivos__

**Nome: Manuella Tavares Vasconcelos**
**Turma: ENGCO221N02**
**Matrícula: 2401790**
---


## __Parte A — Contexto__

Imagine que você está projetando um **sistema de navegação para ambulâncias** em uma cidade.  
Cada interseção é um **nó** e cada rua é uma **aresta ponderada** com o **tempo médio de deslocamento**.

O objetivo é encontrar a **rota mais rápida entre o hospital e o local de atendimento**, garantindo que o caminho tenha **custo mínimo** e seja **correto mesmo em grandes redes urbanas**.

---
Atividades

Implemente o algoritmo de Dijkstra, retornando o custo mínimo (dist) e os predecessores (parent).

Crie uma função reconstruct_path(parent, target) que reconstrua o trajeto.

## __Implementação do Algoritmo de Dijkstra__

In [ ]:
import heapq

def dijkstra(graph, source):
    dist = {node: float('inf') for node in graph}
    parent = {node: None for node in graph}
    dist[source] = 0.0
    heap = [(0.0, source)]
    visited = set()
    while heap:
        d, u = heapq.heappop(heap)
        if u in visited:
            continue
        visited.add(u)
        if d > dist[u]:
            continue
        for v, w in graph[u]:
            if w < 0:
                raise ValueError("Dijkstra requer pesos não negativos nas arestas.")
            nd = d + w
            if nd < dist[v]:
                dist[v] = nd
                parent[v] = u
                heapq.heappush(heap, (nd, v))
    return dist, parent

def reconstruct_path(parent, target):
    path = []
    cur = target
    while cur is not None:
        path.append(cur)
        cur = parent[cur]
    path.reverse()
    return path

#Grafo de Exemplo (Cidade com Hospital e Pontos de Atendimento)

graph = {
    'H': [('B', 4), ('C', 2)],
    'B': [('H', 4), ('C', 1), ('D', 5), ('E', 12)],
    'C': [('H', 2), ('B', 1), ('D', 8)],
    'D': [('B', 5), ('C', 8), ('E', 3), ('F', 7)],
    'E': [('B', 12), ('D', 3), ('F', 4)],
    'F': [('D', 7), ('E', 4), ('A', 6)],
    'A': [('F', 6)]
}

#Execução do Dijkstra e Reconstrução do caminho
dist, parent = dijkstra(graph, 'H')
print('Distâncias calculadas a partir do Hospital (H):')
for k in sorted(dist):
    print(f'{k}: {dist[k]}')
print('\nCaminho H -> A:', reconstruct_path(parent, 'A'), ' | Custo total:', dist['A'])

#Teste de Mudança de Tráfego (Alterar Pesos de Ruas)
#Simulando aumento no tempo entre D e F (de 7 para 50)

graph_mod = {u: [(v,w) for (v,w) in graph[u]] for u in graph}
graph_mod['D'] = [(v, 50 if v=='F' else w) for (v,w) in graph_mod['D']]
graph_mod['F'] = [(v, 50 if v=='D' else w) for (v,w) in graph_mod['F']]

dist2, parent2 = dijkstra(graph_mod, 'H')
print('Novo caminho H -> A:', reconstruct_path(parent2, 'A'), ' | Novo custo:', dist2['A'])


Distâncias calculadas a partir do Hospital (H):
A: 21.0
B: 3.0
C: 2.0
D: 8.0
E: 11.0
F: 15.0
H: 0.0

Caminho H -> A: ['H', 'C', 'B', 'D', 'F', 'A']  | Custo total: 21.0
Novo caminho H -> A: ['H', 'C', 'B', 'D', 'E', 'F', 'A']  | Novo custo: 21.0


## __Questões Discursivas__


### __1️⃣ Por que Dijkstra exige arestas não negativas?__
O algoritmo Dijkstra **assume** que, uma vez encontrado o menor custo até um nó, esse custo **nunca será melhorado** depois.  
Isso só é verdadeiro quando **todos os pesos são ≥ 0**.  
Se existirem pesos negativos, um caminho futuro poderia **diminuir** a distância já calculada, e o resultado estaria errado.  
Por isso, Dijkstra **não funciona com pesos negativos** — para esses casos, usamos o algoritmo de **Bellman-Ford**.

---

### __2️⃣ Qual a complexidade do algoritmo com lista de adjacência e `heapq`?__
A complexidade do algoritmo de Dijkstra usando lista de adjacência e heapq é O(E log V) no tempo e O(V + E) no espaço.
Isso acontece porque o algoritmo examina cada aresta uma vez e usa um heap para escolher o próximo vértice com menor distância, o que custa log V a cada operação.

---

### __3️⃣ Ao testar observei que:__
- Quando aumentei o peso de uma rua (ex: D–F), o caminho ótimo **mudou**, mostrando que o algoritmo reage corretamente às alterações.  
- Quando há empate nos custos, Dijkstra ainda encontra um caminho válido e mínimo.  
- Testes com pesos negativos apresentam um erro, confirmando que a verificação está funcionando.

---


## __Parte B — Contexto__
> Agora, considere um robô autônomo que deve se deslocar por um labirinto 2D, evitando obstáculos e chegando ao destino no menor tempo possível.
Diferente do Dijkstra, o robô pode usar uma heurística (como a distância ao alvo) para priorizar rotas promissoras, economizando tempo de busca.

**Atividades**

1. Gere um grid 20x20 com ~15% de obstáculos aleatórios.

2. Implemente a função heuristic(a, b) (distância Manhattan).

3. Desenvolva o algoritmo a_star(grid, start, goal, h) e teste-o.


In [ ]:
import numpy as np
import random
import heapq

# 1 Gerar grid 20x20 com obstáculos (~15%)
def generate_grid(size=20, obstacle_prob=0.15, seed=None):
    if seed is not None:
        random.seed(seed)
        np.random.seed(seed)
    grid = np.zeros((size, size), dtype=int)
    for i in range(size):
        for j in range(size):
            if random.random() < obstacle_prob:
                grid[i, j] = 1  # 1 = obstáculo
    return grid

# 2️ Heurística Manhattan
def heuristic(a, b):
    # Retorna a soma das diferenças das coordenadas (movimentos ortogonais)
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

# 3️ Algoritmo A*
def a_star(grid, start, goal, h):
    size = grid.shape[0]
    open_set = []
    heapq.heappush(open_set, (0 + h(start, goal), 0, start, [start]))  # (f, g, posição, caminho)
    visited = set()

    while open_set:
        f, g, current, path = heapq.heappop(open_set)

        if current == goal:
            return path, g  # retorna caminho e custo total

        if current in visited:
            continue
        visited.add(current)

        x, y = current
        # Movimentos possíveis: cima, baixo, esquerda, direita
        for dx, dy in [(-1,0),(1,0),(0,-1),(0,1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < size and 0 <= ny < size and grid[nx, ny] == 0:
                neighbor = (nx, ny)
                if neighbor not in visited:
                    new_g = g + 1
                    new_f = new_g + h(neighbor, goal)
                    heapq.heappush(open_set, (new_f, new_g, neighbor, path + [neighbor]))

    return None, float('inf')  # se não houver caminho

# 4️ Teste do A*
grid = generate_grid(20, 0.15, seed=2025)
start = (0, 0)
goal = (19, 19)

path, cost = a_star(grid, start, goal, heuristic)

if path:
    print("Caminho encontrado! Custo:", cost)
    print("Número de passos:", len(path))
else:
    print("Não há caminho possível.")


Caminho encontrado! Custo: 38
Número de passos: 39


## Questões Discursivas


### __1️⃣ A* vs Dijkstra: qual expande menos nós?__

A vs Dijkstra:* A* expande menos nós, pois a heurística prioriza caminhos promissores, enquanto Dijkstra explora todos uniformemente.

---

### __2️⃣ Por que a heurística Manhattan é admissível nesse caso?__
É admissível, porque nunca superestima o custo real; em um grid 2D com movimentos ortogonais, o mínimo de passos até o objetivo é exatamente a soma das diferenças de linhas e colunas.

---
💡 Cenário real: A* ajuda robôs e jogos a encontrar rotas rápidas evitando obstáculos, economizando tempo e energia.


**Parte C — Árvores Binárias e Percursos (DFS em-ordem, pré-ordem e pós-ordem) Contexto**

> Você está desenvolvendo um sistema de recomendação que organiza produtos em uma árvore binária de busca (BST), conforme o preço. Cada nó é um produto e a travessia da árvore pode ser usada para: 1. Ordenar produtos (em-ordem); 2. Clonar a estrutura (pré-ordem); 3. Calcular totais ou liberar memória (pós-ordem);

**Atividades**

1. Crie uma BST com os valores: [50, 30, 70, 20, 40, 60, 80, 35, 45].

Implemente os percursos:

1. in_order(root)

2. pre_order(root)

3. post_order(root)

Teste se as saídas correspondem às travessias esperadas.

In [ ]:
# Definição do nó da árvore
class Node:
    def __init__(self, valor):
        self.valor = valor
        self.esq = None
        self.dir = None

# Inserção na BST
def inserir(root, valor):
    if root is None:
        return Node(valor)
    if valor < root.valor:
        root.esq = inserir(root.esq, valor)
    else:
        root.dir = inserir(root.dir, valor)
    return root

# Percurso em-ordem (esq → raiz → dir)
def in_order(root):
    if root:
        in_order(root.esq)
        print(root.valor, end=' ')
        in_order(root.dir)

# Percurso pré-ordem (raiz → esq → dir)
def pre_order(root):
    if root:
        print(root.valor, end=' ')
        pre_order(root.esq)
        pre_order(root.dir)

# Percurso pós-ordem (esq → dir → raiz)
def post_order(root):
    if root:
        post_order(root.esq)
        post_order(root.dir)
        print(root.valor, end=' ')

# --- Teste ---
valores = [50, 30, 70, 20, 40, 60, 80, 35, 45]
root = None
for v in valores:
    root = inserir(root, v)

print("Em-ordem:")
in_order(root)
print("\nPré-ordem:")
pre_order(root)
print("\nPós-ordem:")
post_order(root)


Em-ordem:
20 30 35 40 45 50 60 70 80 
Pré-ordem:
50 30 20 40 35 45 70 60 80 
Pós-ordem:
20 35 45 40 30 60 80 70 50 

## Questões Discursivas


### __1️⃣Em que situação cada tipo de percurso é mais indicado?__

No percurso em-ordem, a gente visita primeiro o filho da esquerda, depois o nó atual (a raiz) e, por último, o da direita. Esse tipo de travessia é muito útil quando eu quero listar os elementos em ordem crescente, como no caso de produtos organizados por preço.

Já o pré-ordem começa pela raiz, depois passa pela esquerda e pela direita. Ele é bom quando eu quero clonar ou copiar a estrutura da árvore, porque a ordem em que os nós são visitados mantém exatamente o formato original.

Por fim, o pós-ordem visita primeiro a esquerda, depois a direita e só no final o nó raiz. Esse é o mais indicado quando eu preciso calcular totais, liberar memória ou apagar a árvore, já que ele garante que todos os filhos sejam processados antes do nó principal.

## Parte D — Reflexões (Respostas Curtas)

### __1️. Quando não é vantajoso usar A*, mesmo tendo uma heurística?__
O A* não é vantajoso quando o grafo é muito grande, como em mapas de cidades inteiras ou jogos com muitos caminhos possíveis. Se a heurística for fraca (por exemplo, em um mapa, quando a distância estimada é muito diferente da real), o algoritmo perde tempo explorando nós desnecessários, ficando quase tão lento quanto o Dijkstra. Além disso, se calcular a heurística for caro, como estimar distâncias 3D em jogos complexos, o custo de processamento pode anular os benefícios. Também não vale a pena quando os custos mudam com frequência, como em sistemas de tráfego em tempo real.

### __2. Diferencie corretude e otimalidade nos algoritmos estudados.__
A corretude garante que o algoritmo encontre uma solução válida, como achar um caminho possível entre dois pontos em um mapa, mesmo que não seja o mais curto. Já a otimalidade garante que essa solução é a melhor possível — por exemplo, o caminho mais rápido entre duas cidades em um GPS. O Dijkstra e o A* com heurística admissível são corretos e ótimos, pois sempre retornam o menor custo. Em contrapartida, buscas mais simples, como BFS em um mapa com pesos diferentes, podem ser corretas (acham um caminho), mas não ótimas (não é o mais curto).


### __3. Dê um exemplo do mundo real onde cada tipo de percurso (em, pré, pós) é essencial.__

No percurso em-ordem, um sistema de busca de produtos pode exibir itens ordenados por preço, como um e-commerce. O pré-ordem é útil em jogos, quando o sistema precisa copiar o mapa ou recriar uma árvore de decisões dos personagens. Já o pós-ordem é essencial em compiladores, que usam árvores sintáticas para avaliar expressões matemáticas, processando primeiro os operandos e depois o operador. Assim, cada percurso serve para uma necessidade: ordenar, clonar ou processar dados complexos.

### __4. Como heurísticas inconsistentes podem afetar o resultado do A*?__


Heurísticas inconsistentes podem prejudicar o desempenho do A*, especialmente em sistemas de mapas e jogos. Isso acontece quando a estimativa de distância entre os pontos não segue uma lógica estável — por exemplo, quando a “distância aérea” parece menor que a soma dos trechos intermediários. Nesse caso, o algoritmo pode reabrir nós já visitados e recalcular caminhos, ficando mais lento e desperdiçando recursos. Em jogos de estratégia ou sistemas de navegação, isso pode causar trajetos incorretos ou menos eficientes. Por isso, heurísticas consistentes são essenciais para garantir rapidez e resultados ótimos.